In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is 
import sklearn
assert sklearn.__version__ >= "0.20"
from sklearn.model_selection import train_test_split, cross_val_score, KFold, cross_val_predict
from sklearn.linear_model import Perceptron
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, classification_report
from sklearn.metrics import roc_curve, roc_auc_score, f1_score, roc_curve, roc_auc_score 

# Common imports
import numpy as np
import pandas as pd
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [2]:
url= "brain_stroke_in_numeric_without_useless_data.csv"
brain = pd.read_csv(url)
brain = brain.drop(brain.columns[0], axis=1)
X = brain.iloc[:, :8]
y = brain.iloc[:, 8]

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [4]:
per_clf = Perceptron(max_iter=1000, tol=1e-3, random_state=42)
y_pred = per_clf.fit(X_train, y_train).predict(X_test)
print("Accuracy:",accuracy_score(y_test, y_pred))

Accuracy: 0.9478260869565217


In [5]:
kf = KFold(n_splits=10)
score = cross_val_score(per_clf, X_train, y_train, cv=kf, scoring='accuracy')
print("Accuracy Cross Validation score :{}".format(score.mean()))
y_train_pred = cross_val_predict(per_clf, X_train, y_train, cv=kf)
tn, fp, fn, tp = confusion_matrix(y_train, y_train_pred).ravel()
print("TN: %.0f" % tn + ", FP: %.0f" % fp + ", FN: %.0f" % fn + ", TP: %.0f" % tp)
print("Precision:",precision_score(y_train, y_train_pred))
print("Recall:", recall_score(y_train, y_train_pred))
print("f1 score:", f1_score(y_train, y_train_pred))

Accuracy Cross Validation score :0.8125424035833086
TN: 2778, FP: 538, FN: 116, TP: 54
Precision: 0.09121621621621621
Recall: 0.3176470588235294
f1 score: 0.14173228346456693


In [6]:
# if we want to change the number and size of layers we need to use the MLPCClassifier from sklearn

In [7]:
mlp_clf = MLPClassifier(random_state=1)
y_pred = mlp_clf.fit(X_train, y_train).predict(X_test)
print("Accuracy:",accuracy_score(y_test, y_pred))

Accuracy: 0.9491638795986622


In [8]:
kf = KFold(n_splits=10)
score = cross_val_score(mlp_clf, X_train, y_train, cv=kf, scoring='accuracy')
print("Accuracy Cross Validation score :{}".format(score.mean()))

y_train_pred = cross_val_predict(mlp_clf, X_train, y_train, cv=kf)
tn, fp, fn, tp = confusion_matrix(y_train, y_train_pred).ravel()
print("TN: %.0f" % tn + ", FP: %.0f" % fp + ", FN: %.0f" % fn + ", TP: %.0f" % tp)
print("Precision:",precision_score(y_train, y_train_pred))
print("Recall:", recall_score(y_train, y_train_pred))
print("f1 score:", f1_score(y_train, y_train_pred))

Accuracy Cross Validation score :0.9503647531535092
TN: 3313, FP: 3, FN: 170, TP: 0
Precision: 0.0
Recall: 0.0
f1 score: 0.0


In [9]:
# learning rate change 

In [10]:
clf = [
    MLPClassifier(random_state=1, learning_rate_init=0.0001),
    MLPClassifier(random_state=1, learning_rate_init=0.001),
    MLPClassifier(random_state=1, learning_rate_init=0.01),
    MLPClassifier(random_state=1, learning_rate_init=0.1),
    MLPClassifier(random_state=1, learning_rate_init=0.0001, learning_rate='adaptive'),
    MLPClassifier(random_state=1, learning_rate_init=0.001, learning_rate='adaptive'),
    MLPClassifier(random_state=1, learning_rate_init=0.01, learning_rate='adaptive'),
    MLPClassifier(random_state=1, learning_rate_init=0.1, learning_rate='adaptive'),

    ]
clf_columns = []
clf_compare = pd.DataFrame(columns = clf_columns)

row_index = 0
for alg in clf:
        
    predicted = alg.fit(X_train, y_train).predict(X_test)
    kf = KFold(n_splits=10)
    y_train_pred = cross_val_predict(mlp_clf, X_train, y_train, cv=kf)
    clf_name = alg.__class__.__name__
    clf_compare.loc[row_index, 'Accuracy'] = accuracy_score(y_test, predicted )
    tn, fp, fn, tp = confusion_matrix(y_train, y_train_pred).ravel()
    clf_compare.loc[row_index, 'TN'] = tn
    clf_compare.loc[row_index, 'FP'] = fp
    clf_compare.loc[row_index, 'FN'] = fn
    clf_compare.loc[row_index, 'TP'] = tp
    clf_compare.loc[row_index, 'Sensitivity'] = tp / (tp + fn)
    clf_compare.loc[row_index, 'Specificity'] = tn / (tn + fp)
    clf_compare.loc[row_index, 'Precission'] = precision_score(y_train, y_train_pred, zero_division=0)
    clf_compare.loc[row_index, 'Recall'] = recall_score(y_train, y_train_pred, zero_division=0)
    fpr, tpr, thresholds = roc_curve(y_train, y_train_pred)
    clf_compare.loc[row_index, 'AURC'] = roc_auc_score(y_train, y_train_pred)
    row_index+=1
    
clf_compare.sort_values(by = ['Accuracy'], ascending = False, inplace = True)    
clf_compare

,Accuracy,TN,FP,FN,TP,Sensitivity,Specificity,Precission,Recall,AURC
1,0.949164,3313.0,3.0,170.0,0.0,0.0,0.999095,0.0,0.0,0.499548
5,0.949164,3313.0,3.0,170.0,0.0,0.0,0.999095,0.0,0.0,0.499548
0,0.947826,3313.0,3.0,170.0,0.0,0.0,0.999095,0.0,0.0,0.499548
2,0.947826,3313.0,3.0,170.0,0.0,0.0,0.999095,0.0,0.0,0.499548
3,0.947826,3313.0,3.0,170.0,0.0,0.0,0.999095,0.0,0.0,0.499548
4,0.947826,3313.0,3.0,170.0,0.0,0.0,0.999095,0.0,0.0,0.499548
6,0.947826,3313.0,3.0,170.0,0.0,0.0,0.999095,0.0,0.0,0.499548
7,0.947826,3313.0,3.0,170.0,0.0,0.0,0.999095,0.0,0.0,0.499548


In [11]:
# number of layers and size of layers

In [12]:
clf = [
    MLPClassifier(random_state=1, hidden_layer_sizes=(6,4,2,1)),
    MLPClassifier(random_state=1, hidden_layer_sizes=(4,2,1)),
    MLPClassifier(random_state=1, hidden_layer_sizes=(4,2,)),
    MLPClassifier(random_state=1, hidden_layer_sizes=(2,)),
    ]
clf_columns = []
clf_compare = pd.DataFrame(columns = clf_columns)

row_index = 0
for alg in clf:
        
    predicted = alg.fit(X_train, y_train).predict(X_test)
    kf = KFold(n_splits=10)
    y_train_pred = cross_val_predict(mlp_clf, X_train, y_train, cv=kf)
    clf_name = alg.__class__.__name__
    clf_compare.loc[row_index, 'Accuracy'] = accuracy_score(y_test, predicted )
    tn, fp, fn, tp = confusion_matrix(y_train, y_train_pred).ravel()
    clf_compare.loc[row_index, 'TN'] = tn
    clf_compare.loc[row_index, 'FP'] = fp
    clf_compare.loc[row_index, 'FN'] = fn
    clf_compare.loc[row_index, 'TP'] = tp
    clf_compare.loc[row_index, 'Sensitivity'] = tp / (tp + fn)
    clf_compare.loc[row_index, 'Specificity'] = tn / (tn + fp)
    clf_compare.loc[row_index, 'Precission'] = precision_score(y_train, y_train_pred, zero_division=0)
    clf_compare.loc[row_index, 'Recall'] = recall_score(y_train, y_train_pred, zero_division=0)
    fpr, tpr, thresholds = roc_curve(y_train, y_train_pred)
    clf_compare.loc[row_index, 'AURC'] = roc_auc_score(y_train, y_train_pred)
    row_index+=1
    
clf_compare.sort_values(by = ['Accuracy'], ascending = False, inplace = True)    
clf_compare

/home/lia/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/lia/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/lia/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


,Accuracy,TN,FP,FN,TP,Sensitivity,Specificity,Precission,Recall,AURC
0,0.947826,3313.0,3.0,170.0,0.0,0.0,0.999095,0.0,0.0,0.499548
1,0.947826,3313.0,3.0,170.0,0.0,0.0,0.999095,0.0,0.0,0.499548
2,0.947826,3313.0,3.0,170.0,0.0,0.0,0.999095,0.0,0.0,0.499548
3,0.947826,3313.0,3.0,170.0,0.0,0.0,0.999095,0.0,0.0,0.499548


In [13]:
# number of iterations

In [14]:
clf = [
    MLPClassifier(random_state=1, max_iter=400),
    MLPClassifier(random_state=1, max_iter=300),
    MLPClassifier(random_state=1, max_iter=200),
    MLPClassifier(random_state=1, max_iter=100),
    MLPClassifier(random_state=1, max_iter=50),
    MLPClassifier(random_state=1, max_iter=10),
    

    ]
clf_columns = []
clf_compare = pd.DataFrame(columns = clf_columns)

row_index = 0
for alg in clf:
        
    predicted = alg.fit(X_train, y_train).predict(X_test)
    kf = KFold(n_splits=10)
    y_train_pred = cross_val_predict(mlp_clf, X_train, y_train, cv=kf)
    clf_name = alg.__class__.__name__
    clf_compare.loc[row_index, 'Accuracy'] = accuracy_score(y_test, predicted )
    tn, fp, fn, tp = confusion_matrix(y_train, y_train_pred).ravel()
    clf_compare.loc[row_index, 'TN'] = tn
    clf_compare.loc[row_index, 'FP'] = fp
    clf_compare.loc[row_index, 'FN'] = fn
    clf_compare.loc[row_index, 'TP'] = tp
    clf_compare.loc[row_index, 'Sensitivity'] = tp / (tp + fn)
    clf_compare.loc[row_index, 'Specificity'] = tn / (tn + fp)
    clf_compare.loc[row_index, 'Precission'] = precision_score(y_train, y_train_pred, zero_division=0)
    clf_compare.loc[row_index, 'Recall'] = recall_score(y_train, y_train_pred, zero_division=0)
    fpr, tpr, thresholds = roc_curve(y_train, y_train_pred)
    clf_compare.loc[row_index, 'AURC'] = roc_auc_score(y_train, y_train_pred)
    row_index+=1
    
clf_compare.sort_values(by = ['Accuracy'], ascending = False, inplace = True)    
clf_compare

/home/lia/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/lia/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/lia/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


,Accuracy,TN,FP,FN,TP,Sensitivity,Specificity,Precission,Recall,AURC
0,0.949164,3313.0,3.0,170.0,0.0,0.0,0.999095,0.0,0.0,0.499548
1,0.949164,3313.0,3.0,170.0,0.0,0.0,0.999095,0.0,0.0,0.499548
2,0.949164,3313.0,3.0,170.0,0.0,0.0,0.999095,0.0,0.0,0.499548
3,0.947826,3313.0,3.0,170.0,0.0,0.0,0.999095,0.0,0.0,0.499548
4,0.947826,3313.0,3.0,170.0,0.0,0.0,0.999095,0.0,0.0,0.499548
5,0.947826,3313.0,3.0,170.0,0.0,0.0,0.999095,0.0,0.0,0.499548


In [15]:
# batch size

In [16]:
clf = [
    MLPClassifier(random_state=1, batch_size=min(200, 70)),
    MLPClassifier(random_state=1, batch_size=min(200, 50)),
    MLPClassifier(random_state=1, batch_size=min(200, 20)),
    MLPClassifier(random_state=1, batch_size=min(200, 10)),
    MLPClassifier(random_state=1, batch_size=min(200, 5)),
    ]
clf_columns = []
clf_compare = pd.DataFrame(columns = clf_columns)

row_index = 0
for alg in clf:
        
    predicted = alg.fit(X_train, y_train).predict(X_test)
    kf = KFold(n_splits=10)
    y_train_pred = cross_val_predict(mlp_clf, X_train, y_train, cv=kf)
    clf_name = alg.__class__.__name__
    clf_compare.loc[row_index, 'Accuracy'] = accuracy_score(y_test, predicted )
    tn, fp, fn, tp = confusion_matrix(y_train, y_train_pred).ravel()
    clf_compare.loc[row_index, 'TN'] = tn
    clf_compare.loc[row_index, 'FP'] = fp
    clf_compare.loc[row_index, 'FN'] = fn
    clf_compare.loc[row_index, 'TP'] = tp
    clf_compare.loc[row_index, 'Sensitivity'] = tp / (tp + fn)
    clf_compare.loc[row_index, 'Specificity'] = tn / (tn + fp)
    clf_compare.loc[row_index, 'Precission'] = precision_score(y_train, y_train_pred, zero_division=0)
    clf_compare.loc[row_index, 'Recall'] = recall_score(y_train, y_train_pred, zero_division=0)
    fpr, tpr, thresholds = roc_curve(y_train, y_train_pred)
    clf_compare.loc[row_index, 'AURC'] = roc_auc_score(y_train, y_train_pred)
    row_index+=1
    
clf_compare.sort_values(by = ['Accuracy'], ascending = False, inplace = True)    
clf_compare

,Accuracy,TN,FP,FN,TP,Sensitivity,Specificity,Precission,Recall,AURC
3,0.948495,3313.0,3.0,170.0,0.0,0.0,0.999095,0.0,0.0,0.499548
0,0.947826,3313.0,3.0,170.0,0.0,0.0,0.999095,0.0,0.0,0.499548
1,0.947826,3313.0,3.0,170.0,0.0,0.0,0.999095,0.0,0.0,0.499548
4,0.947826,3313.0,3.0,170.0,0.0,0.0,0.999095,0.0,0.0,0.499548
2,0.912375,3313.0,3.0,170.0,0.0,0.0,0.999095,0.0,0.0,0.499548


In [17]:
# momentum

In [18]:
clf = [
    MLPClassifier(random_state=1, solver='sgd', momentum=0.9),
    MLPClassifier(random_state=1, solver='sgd', momentum=0.7),
    MLPClassifier(random_state=1, solver='sgd', momentum=0.5),
    MLPClassifier(random_state=1, solver='sgd', momentum=0.3),
    MLPClassifier(random_state=1, solver='sgd', momentum=0.1),
    

    ]
clf_columns = []
clf_compare = pd.DataFrame(columns = clf_columns)

row_index = 0
for alg in clf:
        
    predicted = alg.fit(X_train, y_train).predict(X_test)
    kf = KFold(n_splits=10)
    y_train_pred = cross_val_predict(mlp_clf, X_train, y_train, cv=kf)
    clf_name = alg.__class__.__name__
    clf_compare.loc[row_index, 'Accuracy'] = accuracy_score(y_test, predicted )
    tn, fp, fn, tp = confusion_matrix(y_train, y_train_pred).ravel()
    clf_compare.loc[row_index, 'TN'] = tn
    clf_compare.loc[row_index, 'FP'] = fp
    clf_compare.loc[row_index, 'FN'] = fn
    clf_compare.loc[row_index, 'TP'] = tp
    clf_compare.loc[row_index, 'Sensitivity'] = tp / (tp + fn)
    clf_compare.loc[row_index, 'Specificity'] = tn / (tn + fp)
    clf_compare.loc[row_index, 'Precission'] = precision_score(y_train, y_train_pred, zero_division=0)
    clf_compare.loc[row_index, 'Recall'] = recall_score(y_train, y_train_pred, zero_division=0)
    fpr, tpr, thresholds = roc_curve(y_train, y_train_pred)
    clf_compare.loc[row_index, 'AURC'] = roc_auc_score(y_train, y_train_pred)
    row_index+=1
    
clf_compare.sort_values(by = ['Accuracy'], ascending = False, inplace = True)    
clf_compare

,Accuracy,TN,FP,FN,TP,Sensitivity,Specificity,Precission,Recall,AURC
0,0.931104,3313.0,3.0,170.0,0.0,0.0,0.999095,0.0,0.0,0.499548
4,0.912375,3313.0,3.0,170.0,0.0,0.0,0.999095,0.0,0.0,0.499548
3,0.892308,3313.0,3.0,170.0,0.0,0.0,0.999095,0.0,0.0,0.499548
2,0.871572,3313.0,3.0,170.0,0.0,0.0,0.999095,0.0,0.0,0.499548
1,0.867559,3313.0,3.0,170.0,0.0,0.0,0.999095,0.0,0.0,0.499548


In [19]:
# solver

In [20]:
clf = [
    MLPClassifier(random_state=1, solver='sgd'),
    MLPClassifier(random_state=1, solver='lbfgs'),
    MLPClassifier(random_state=1, solver='adam'),

    ]
clf_columns = []
clf_compare = pd.DataFrame(columns = clf_columns)

row_index = 0
for alg in clf:
        
    predicted = alg.fit(X_train, y_train).predict(X_test)
    kf = KFold(n_splits=10)
    y_train_pred = cross_val_predict(mlp_clf, X_train, y_train, cv=kf)
    clf_name = alg.__class__.__name__
    clf_compare.loc[row_index, 'Accuracy'] = accuracy_score(y_test, predicted )
    tn, fp, fn, tp = confusion_matrix(y_train, y_train_pred).ravel()
    clf_compare.loc[row_index, 'TN'] = tn
    clf_compare.loc[row_index, 'FP'] = fp
    clf_compare.loc[row_index, 'FN'] = fn
    clf_compare.loc[row_index, 'TP'] = tp
    clf_compare.loc[row_index, 'Sensitivity'] = tp / (tp + fn)
    clf_compare.loc[row_index, 'Specificity'] = tn / (tn + fp)
    clf_compare.loc[row_index, 'Precission'] = precision_score(y_train, y_train_pred, zero_division=0)
    clf_compare.loc[row_index, 'Recall'] = recall_score(y_train, y_train_pred, zero_division=0)
    fpr, tpr, thresholds = roc_curve(y_train, y_train_pred)
    clf_compare.loc[row_index, 'AURC'] = roc_auc_score(y_train, y_train_pred)
    row_index+=1
    
clf_compare.sort_values(by = ['Accuracy'], ascending = False, inplace = True)    
clf_compare

/home/lia/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


,Accuracy,TN,FP,FN,TP,Sensitivity,Specificity,Precission,Recall,AURC
2,0.949164,3313.0,3.0,170.0,0.0,0.0,0.999095,0.0,0.0,0.499548
1,0.947826,3313.0,3.0,170.0,0.0,0.0,0.999095,0.0,0.0,0.499548
0,0.931104,3313.0,3.0,170.0,0.0,0.0,0.999095,0.0,0.0,0.499548


In [21]:
# activation function

In [22]:
clf = [
    MLPClassifier(random_state=1, activation='identity'),
    MLPClassifier(random_state=1, activation='logistic'),
    MLPClassifier(random_state=1, activation='tanh'),
    MLPClassifier(random_state=1, activation='relu'),

    ]
clf_columns = []
clf_compare = pd.DataFrame(columns = clf_columns)

row_index = 0
for alg in clf:
        
    predicted = alg.fit(X_train, y_train).predict(X_test)
    kf = KFold(n_splits=10)
    y_train_pred = cross_val_predict(mlp_clf, X_train, y_train, cv=kf)
    clf_name = alg.__class__.__name__
    clf_compare.loc[row_index, 'Accuracy'] = accuracy_score(y_test, predicted )
    tn, fp, fn, tp = confusion_matrix(y_train, y_train_pred).ravel()
    clf_compare.loc[row_index, 'TN'] = tn
    clf_compare.loc[row_index, 'FP'] = fp
    clf_compare.loc[row_index, 'FN'] = fn
    clf_compare.loc[row_index, 'TP'] = tp
    clf_compare.loc[row_index, 'Sensitivity'] = tp / (tp + fn)
    clf_compare.loc[row_index, 'Specificity'] = tn / (tn + fp)
    clf_compare.loc[row_index, 'Precission'] = precision_score(y_train, y_train_pred, zero_division=0)
    clf_compare.loc[row_index, 'Recall'] = recall_score(y_train, y_train_pred, zero_division=0)
    fpr, tpr, thresholds = roc_curve(y_train, y_train_pred)
    clf_compare.loc[row_index, 'AURC'] = roc_auc_score(y_train, y_train_pred)
    row_index+=1
    
clf_compare.sort_values(by = ['Accuracy'], ascending = False, inplace = True)    
clf_compare

,Accuracy,TN,FP,FN,TP,Sensitivity,Specificity,Precission,Recall,AURC
3,0.949164,3313.0,3.0,170.0,0.0,0.0,0.999095,0.0,0.0,0.499548
0,0.947826,3313.0,3.0,170.0,0.0,0.0,0.999095,0.0,0.0,0.499548
1,0.947826,3313.0,3.0,170.0,0.0,0.0,0.999095,0.0,0.0,0.499548
2,0.947826,3313.0,3.0,170.0,0.0,0.0,0.999095,0.0,0.0,0.499548


In [23]:
# best parameters :
# MLPClassifier(random_state=1, learning_rate_init=0.001),
# MLPClassifier(random_state=1, hidden_layer_sizes=(4,2,1)),
# MLPClassifier(random_state=1, max_iter=200),
# MLPClassifier(random_state=1, batch_size=min(200, 10)),
# MLPClassifier(random_state=1, solver='sgd', momentum=0.9),
# MLPClassifier(random_state=1, solver='adam'),
# MLPClassifier(random_state=1, activation='relu'),

In [24]:
# Classifier 1

In [25]:
X = brain.iloc[:, :8]
y = brain.iloc[:, 8]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [26]:
mlp_clf = MLPClassifier(random_state=1, learning_rate_init=0.001, max_iter=200, batch_size=min(200, 10), solver='adam', activation='relu')
y_pred = mlp_clf.fit(X_train, y_train).predict(X_test)
print("Accuracy:",accuracy_score(y_test, y_pred))

Accuracy: 0.948494983277592


In [27]:
y_train_pred = cross_val_predict(mlp_clf, X_train, y_train, cv=kf)
tn, fp, fn, tp = confusion_matrix(y_train, y_train_pred).ravel()
print("TN: %.0f" % tn + ", FP: %.0f" % fp + ", FN: %.0f" % fn + ", TP: %.0f" % tp)
print("Precision:",precision_score(y_train, y_train_pred))
print("Recall:", recall_score(y_train, y_train_pred))
print("Sensitivity:", tp / (tp + fn))
print("Specificity:", tn / (tn + fp))
print("AURC:", roc_auc_score(y_train, y_train_pred))

TN: 3313, FP: 3, FN: 169, TP: 1
Precision: 0.25
Recall: 0.0058823529411764705
Sensitivity: 0.0058823529411764705
Specificity: 0.9990952955367913
AURC: 0.5024888242389839


In [28]:
# Classifier 2

In [29]:
X = brain.iloc[:, :8]
y = brain.iloc[:, 8]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
mlp_clf = MLPClassifier(random_state=1, learning_rate_init=0.001, max_iter=200, batch_size=min(200, 10), solver='adam', activation='relu')
y_pred = mlp_clf.fit(X_train, y_train).predict(X_test)
print("Accuracy:",accuracy_score(y_test, y_pred))

Accuracy: 0.948494983277592


In [30]:
y_train_pred = cross_val_predict(mlp_clf, X_train, y_train, cv=kf)
tn, fp, fn, tp = confusion_matrix(y_train, y_train_pred).ravel()
print("TN: %.0f" % tn + ", FP: %.0f" % fp + ", FN: %.0f" % fn + ", TP: %.0f" % tp)
print("Precision:",precision_score(y_train, y_train_pred))
print("Recall:", recall_score(y_train, y_train_pred))
print("Sensitivity:", tp / (tp + fn))
print("Specificity:", tn / (tn + fp))
print("AURC:", roc_auc_score(y_train, y_train_pred))

TN: 3313, FP: 3, FN: 169, TP: 1
Precision: 0.25
Recall: 0.0058823529411764705
Sensitivity: 0.0058823529411764705
Specificity: 0.9990952955367913
AURC: 0.5024888242389839


In [31]:
# Classifier 3

In [32]:
X = brain.iloc[:, :8]
y = brain.iloc[:, 8]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=42)
mlp_clf = MLPClassifier(random_state=1, learning_rate_init=0.001, max_iter=200, batch_size=min(200, 10), solver='adam', activation='relu')
y_pred = mlp_clf.fit(X_train, y_train).predict(X_test)
print("Accuracy:",accuracy_score(y_test, y_pred))

Accuracy: 0.9521579123452659


In [33]:
y_train_pred = cross_val_predict(mlp_clf, X_train, y_train, cv=kf)
tn, fp, fn, tp = confusion_matrix(y_train, y_train_pred).ravel()
print("TN: %.0f" % tn + ", FP: %.0f" % fp + ", FN: %.0f" % fn + ", TP: %.0f" % tp)
print("Precision:",precision_score(y_train, y_train_pred))
print("Recall:", recall_score(y_train, y_train_pred))
print("Sensitivity:", tp / (tp + fn))
print("Specificity:", tn / (tn + fp))
print("AURC:", roc_auc_score(y_train, y_train_pred))

TN: 1867, FP: 20, FN: 104, TP: 1
Precision: 0.047619047619047616
Recall: 0.009523809523809525
Sensitivity: 0.009523809523809525
Specificity: 0.9894011658717541
AURC: 0.4994624876977818
